In [1]:
import numpy as np
from sklearn.metrics import f1_score, make_scorer, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from backbone.probability_transformer import ProbabilityTransformer 
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from backbone.utils import load_function
from typing import Tuple
import yaml
from sklearn.metrics import classification_report
from datetime import datetime, timedelta

In [2]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)

periods_forward = 10

tickers = ['EURUSD', 'GBPUSD', 'USDJPY', 'USDCAD', 'AUDUSD', 'USDCHF']
symbols_path = './backbone/data/backtest/symbols'
instruments = {}
df = pd.DataFrame()

for ticker in tickers:
    instruments[ticker] = pd.read_csv(os.path.join(symbols_path, f'{ticker}.csv'))
  
    instruments[ticker]['ticker'] = ticker
  
    print('Creando target')
    instruments[ticker] = instruments[ticker].sort_values(by='Date')
    instruments[ticker]['target'] = ((instruments[ticker]['Close'].shift(-periods_forward) - instruments[ticker]['Close']) / instruments[ticker]['Close']) * 100
    
    cut_right = round(instruments[ticker]['target'].mean() + 1 * instruments[ticker]['target'].std(), 2)
    cut_left = round(instruments[ticker]['target'].mean() - 1 * instruments[ticker]['target'].std(), 2)
    
    bins = [-100000, cut_left, cut_right, 100000]
    labels = [0, 1, 2]

    instruments[ticker]['target'] = pd.cut(instruments[ticker]['target'], bins, labels=labels)
    
    df = pd.concat([
        df,
        instruments[ticker]
    ])

    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d %H:00:00')

    df = df.sort_values(by='Date')

Creando target
Creando target
Creando target
Creando target
Creando target
Creando target


In [8]:
import talib

In [15]:
integer = talib.CDLCLOSINGMARUBOZU(df.Open, df.High, df.Low, df.Close)
integer.value_counts()

 0      41744
 100      238
-100      203
Name: count, dtype: int64

In [16]:
integer = talib.CDLDOJI(df.Open, df.High, df.Low, df.Close)
integer.value_counts()

100    34488
0       7697
Name: count, dtype: int64

In [17]:
integer = talib.CDLDOJISTAR (df.Open, df.High, df.Low, df.Close)
integer.value_counts()

 0      39135
 100     3022
-100       28
Name: count, dtype: int64

In [18]:
integer = talib.CDLENGULFING  (df.Open, df.High, df.Low, df.Close)
integer.value_counts()

 0      42077
-100       57
 100       51
Name: count, dtype: int64

In [3]:
date_format = '%Y-%m-%d %H:00:00'
window = 1440

actual_date = datetime(2023,5,1,0,0,0)

date_to = actual_date - timedelta(hours=periods_forward+1) 
date_from = date_to - timedelta(hours=window)

date_from_test = actual_date
date_to_test = date_from_test + timedelta(hours=48)

date_from_str = date_from.strftime(date_format)
date_to_str = date_to.strftime(date_format)
date_from_test_str = date_from_test.strftime(date_format)
date_to_test_str = date_to_test.strftime(date_format)


train = df[(df['Date']>date_from_str) & (df['Date']<date_to_str)]
test = df[(df['Date']>date_from_test_str) & (df['Date']<date_to_test_str)]

# Inicio undersampling
class_0 = train[train['target']==0]
class_2 = train[train['target']==2]
avg_examples = (class_0.shape[0] + class_2.shape[0]) / 2
class_1 = train[train['target']==1].tail(int(avg_examples)).sample(frac=1)

train = pd.concat([class_0, class_1, class_2])
# fin undersampling

print(train.Date.head(1))
print(train.Date.tail(1))
print('------------------------')
print(test.Date.head(1))
print(test.Date.tail(1))

train.target.value_counts()

1838   2023-03-01 14:00:00
Name: Date, dtype: datetime64[ns]
2853   2023-04-28 23:00:00
Name: Date, dtype: datetime64[ns]
------------------------
2855   2023-05-01 01:00:00
Name: Date, dtype: datetime64[ns]
2901   2023-05-02 23:00:00
Name: Date, dtype: datetime64[ns]


target
0    785
1    777
2    770
Name: count, dtype: int64

In [4]:
scaler = StandardScaler()
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model = XGBClassifier()

pipe = Pipeline([
    ('scaler', scaler),
    ('prob_transf', ProbabilityTransformer(model)),
    ('log_reg', log_reg)
])

with open('configs/model_config.yml', 'r') as file:
    model_configs = yaml.safe_load(file)

param_grid = {
    'prob_transf__model__objective': ['multi:softprob'],
    'prob_transf__model__max_depth': [2, 5, 8],
    'prob_transf__model__n_estimators': [5, 8],
    'prob_transf__model__learning_rate': [0.05, 0.01, 0.1],
    'prob_transf__model__random_state': [42]
}

n_splits = 5
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

search = GridSearchCV(
    pipe,
    param_grid,
    n_jobs=-1,
    cv=stratified_kfold,
    scoring=make_scorer(precision_score, average='weighted')
)

search.fit(train.drop(columns=['target', 'Date', 'ticker']), train.target)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('prob_transf',
                                        ProbabilityTransformer(model=XGBClassifier(base_score=None,
                                                                                   booster=None,
                                                                                   callbacks=None,
                                                                                   colsample_bylevel=None,
                                                                                   colsample_bynode=None,
                                                                                   colsample_bytree=None,
                                                                                   device=None,
                                                                                   early_stopping_rounds=None,
                                                                                   enable_categori...
                                        LogisticRegression(max_iter=1000,
                                                           multi_class='multinomial'))]),
             n_jobs=-1,
             param_grid={'prob_transf__model__learning_rate': [0.05, 0.01, 0.1],
                         'prob_transf__model__max_depth': [2, 5, 8],
                         'prob_transf__model__n_estimators': [5, 8],
                         'prob_transf__model__objective': ['multi:softprob'],
                         'prob_transf__model__random_state': [42]},
             scoring=make_scorer(precision_score, average=weighted))

In [19]:
x = train.drop(columns=['target', 'Date', 'ticker'])
y = train.target

In [22]:
y.value_counts()

target
0    785
1    777
2    770
Name: count, dtype: int64

In [5]:
pipeline = search.best_estimator_

predictions = pipeline.predict_proba(train.drop(columns=['target', 'Date', 'ticker']))
max_probabilities = np.max(predictions, axis=1)
max_indices = np.argmax(predictions, axis=1)

precision = precision_score(train.target, max_indices, average='weighted')
recall = recall_score(train.target, max_indices, average='weighted')
f1 = f1_score(train.target, max_indices, average='weighted')

print(precision)
print(recall)
print(f1) 

target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(train.target, max_indices, target_names=target_names))

0.9303794219142933
0.9275300171526587
0.9274541117418997
              precision    recall  f1-score   support

     class 0       0.95      0.91      0.93       785
     class 1       0.88      0.98      0.93       777
     class 2       0.96      0.89      0.92       770

    accuracy                           0.93      2332
   macro avg       0.93      0.93      0.93      2332
weighted avg       0.93      0.93      0.93      2332



In [6]:
predictions = pipeline.predict_proba(test.drop(columns=['target', 'Date', 'ticker']))
max_probabilities = np.max(predictions, axis=1)
max_indices = np.argmax(predictions, axis=1)

precision = precision_score(test.target, max_indices, average='weighted')
recall = recall_score(test.target, max_indices, average='weighted')
f1 = f1_score(test.target, max_indices, average='weighted')

print(precision)
print(recall)
print(f1)

target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(test.target, max_indices, target_names=target_names))

0.047833148346721345
0.1595744680851064
0.07135581711181531
              precision    recall  f1-score   support

     class 0       0.15      0.72      0.25        40
     class 1       0.00      0.00      0.00       201
     class 2       0.18      0.39      0.25        41

    accuracy                           0.16       282
   macro avg       0.11      0.37      0.17       282
weighted avg       0.05      0.16      0.07       282



c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` par